# Aimes, Iowa Housing Data: Preprocessing and Model Fitting (Intuitive Interactions, Linear Regression)

In this notebook, we process our data further before fitting it into a model.  The data is split into training and testing sets, scaled and fit.  Predictors are guessed through previous EDA and fitting more area variables, though these are admittedly guessed toward.  We also note that there was skew in the distributions, but a linear regression is used in this manner to interpret large scale trends.

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, LassoCV
import patsy
from sklearn.metrics import mean_squared_error

import pickle
from functools import reduce
from itertools import combinations
np.random.seed(1)

In [2]:
fileObj = open('./pickles/housingDF2.pkl', 'rb')
housing = pickle.load(fileObj)
fileObj.close()

In this notebook, we further explore modelling on a minimal set of interactions.  Lot frontage may have had a more obvious interaction with lot configuration, but these were discarded earlier due to missing information.  Imputation may have had biased results.

In [3]:
areas = housing.loc[:,housing.columns.map(lambda name: 'SF' in name or 'Area' in name or 'Porch' in name)].columns
quals = housing.loc[:,housing.columns.map(lambda name: 'Qual' in name or 'QC' in name)].columns
types = housing.loc[:,housing.columns.map(lambda name: 'Type' in name)].columns

In [4]:
area_zero = []
for area in areas:
    housing[area + 'HasZeros'] = housing[area].map(lambda x: int(x>0))
    area_zero.append(area+'HasZeros')

In [5]:
area_splits = zip(areas, area_zero)

In [6]:
area_strings = []
for row in list(area_splits):
    area_strings.append(row[0] + ':' + row[1])

In [7]:
area_strings = reduce((lambda x,y: x + ' + ' + y), area_strings)

In [8]:
areas = reduce((lambda x,y: x + ' + ' + y), areas)
quals = reduce((lambda x,y: x + ' + ' + y), quals)

In [9]:
type_combos = list(combinations(types,2))

In [10]:
type_strings = []
for combo in type_combos:
    type_strings.append(combo[0] + ':' + combo[1])
    
type_strings = reduce((lambda x,y: x + ' + ' + y), type_strings)

In [11]:
terms = reduce((lambda x,y: x + ' + ' + y),housing.drop(['SalePrice'], axis=1).columns)
interactions = 'TotalBsmtSF:BsmtExposure + LotArea:LotConfig + GarageArea:GarageFinish + OverallQual:GrLivArea'
formula = f'SalePrice ~ {terms} + ({area_strings}):({quals}) - 1'
y, x = patsy.dmatrices(formula, housing)
x = pd.DataFrame(x, columns=x.design_info.column_names)

In [59]:
model = LinearRegression()
xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.7, shuffle=True)

#scaler = StandardScaler()
#xtrain = scaler.fit_transform(xtrain)
#xtest = scaler.transform(xtest)

In [16]:
model = LassoCV()
xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.7)

scaler = StandardScaler()
xtrain = scaler.fit_transform(xtrain)
xtest = scaler.transform(xtest)

#ytrain = np.log(ytrain)[:,0]
#ytest = np.log(ytest)[:,0]

In [17]:
model.fit(xtrain, ytrain)
print(f'The model\'s R^2 score is {model.score(xtrain, ytrain)} on the training data, {model.score(xtest, ytest)} on the test data.')

/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:1094: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


The model's R^2 score is 0.9625461283132662 on the training data, 0.7605423106936764 on the test data.


In [18]:
mean_squared_error(ytest, model.predict(xtest))**0.5

38300.79759569337

This model scores somewhat consistently in cross validation, and the largest weight is from quality area(OverallQual:GrLivArea), as promised.  We see that garage area has a large negative coefficient for garages of type 0(those that were listed as having no garage) to conform to the data, but making no sense if we try to infer a relationship from this.

In [15]:
pd.Series(model.coef_, index=x.columns).sort_values(ascending=False)

GrLivArea:GrLivAreaHasZeros:OverallQual                21408.607472
GrLivArea                                              10837.493232
TotalBsmtSF:TotalBsmtSFHasZeros:OverallQual             8531.703669
GarageArea:GarageAreaHasZeros:OverallQual               7749.719090
YearBuilt                                               7189.550858
BsmtFinSF1:BsmtFinSF1HasZeros:OverallQual               6894.840921
LotArea:LotAreaHasZeros:OverallQual                     5382.135473
BsmtFinSF2:BsmtFinSF2HasZeros:BsmtQual[T.Ex]            4467.328577
OverallCond                                             4278.247311
GrLivArea:GrLivAreaHasZeros:BsmtQual[T.Ex]              3795.696670
TotalBsmtSF:TotalBsmtSFHasZeros:ExterQual[Ex]           3509.168405
LotArea:LotAreaHasZeros:ExterQual[Ex]                   3221.328573
Exterior1st[T.BrkFace]                                  2908.289059
MasVnrArea:MasVnrAreaHasZeros:ExterQual[Ex]             2611.688597
Functional[T.Typ]                               

In [77]:
scores = cross_val_score(model, xtrain, ytrain.ravel(), cv=10)
print(f'With 10 folds, the R^2 score is {np.mean(scores)} +- {np.std(scores)}')

With 10 folds, the R^2 score is 0.9230054925190355 +- 0.016450176283352044


This may be just by virtue of poorly choosing parameters: in fact, we get similar results by applying a linear combination of overall quality and living area rather than the product.

In [9]:
interactions = 'TotalBsmtSF:BsmtExposure + LotArea:LotConfig + GarageArea:GarageFinish + OverallQual + GrLivArea'
formula = f'SalePrice ~ {interactions} - 1'
y, x = patsy.dmatrices(formula, housing)
x = pd.DataFrame(x, columns=x.design_info.column_names)

In [10]:
model = LinearRegression()
xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.7, shuffle=True)

scaler = StandardScaler()
xtrain = scaler.fit_transform(xtrain)
xtest = scaler.transform(xtest)

In [11]:
model.fit(xtrain, ytrain)
print(f'The model\'s R^2 score is {model.score(xtrain, ytrain)} on the training data, {model.score(xtest, ytest)} on the test data.')

The model's R^2 score is 0.8686897681690234 on the training data, 0.8430454758047368 on the test data.


In [12]:
scores = cross_val_score(model, xtrain, ytrain.ravel(), cv=10)
print(f'With 10 folds, the R^2 score is {np.mean(scores)} +- {np.std(scores)}')

With 10 folds, the R^2 score is 0.8518301924838829 +- 0.030771909533508765


Rather than guessing for features, we should automate the process.  We first try this using a Lasso regression model.